In [1]:
# Install Keras Tuner if not already installed
# pip install keras-tuner

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define the model-building function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))

    # Tune the number of hidden layers and their units
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                               activation='relu'))

    model.add(layers.Dense(10, activation='softmax'))

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Choose the optimizer
    optimizer_name = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    optimizer = None

    if optimizer_name == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(learning_rate=hp_learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=hp_learning_rate)

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Instantiate the tuner and perform the hyperparameter search
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=2,  # Number of hyperparameter combinations to try
                     directory='tuner_dir',  # Directory to save results
                     project_name='mnist_tuning')  # Project name

# Search for the best hyperparameter configuration
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Get the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print('Best Hyperparameters:')
print(best_hyperparameters)

# Evaluate the best model on the test set
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print('\nTest accuracy:', test_acc)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6568\36682508.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Reloading Tuner from tuner_dir\mnist_tuning\tuner0.json
Best Hyperparameters:
313/313 [==============================] - 1s 2ms/step - loss: 0.0913 - accuracy: 0.9767

Test accuracy: 0.9767000079154968
